In [1]:
from datetime import datetime, timedelta
import re
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime

In [2]:
# Create an Engine object
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

In [3]:
chromedriver_path = "C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe"
url = 'https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin'

In [4]:
def login(path=None):
    if path == None:
        path = r"C:\ProgramData\Anaconda3\WebDriver\bin\chromedriver.exe"
    service = Service(executable_path=path) # Here
    driver = webdriver.Chrome(service=service) # Here
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    login_form = driver.find_element(By.ID, 'name')
    login_form.send_keys('gostros09')
    login_form = driver.find_element(By.NAME, 'password')
    login_form.send_keys('Segneri9A')
    continue_link = driver.find_element(By.TAG_NAME, 'BUTTON')
    driver.implicitly_wait(3)
    continue_link.click()
    return driver

In [5]:
def cbs_login():
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service)
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')

    login_form = driver.find_element(By.ID, 'name')
    login_form.send_keys('gostros09')
    login_form = driver.find_element(By.NAME, 'password')
    login_form.send_keys('Segneri9A')
    time.sleep(15)
    # Wait until the "Continue" button is present and clickable
    continue_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continue')]"))
    )
    # Click the "Continue" button
    continue_button.click()

    # Additional actions can be performed here    
    return driver

In [6]:
from scraper import Scraper
s = Scraper()

In [7]:
df = s.cbs_auction_values()

C:\Users\pddnh\Documents\GitHub\xdl\scraper.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=1, skiprows=0, extract_links='body')[0]  # [0] to get the first DataFrame if there are multiple


ValueError: No tables found

In [ ]:
df

### Manual approach

In [8]:
driver = cbs_login()

In [9]:
driver.get('https://xdl.baseball.cbssports.com/features/projected-salaries')

In [10]:
destination_path = r"C:\Users\pddnh\Documents\GitHub\xdl\data"

In [11]:
# Get the HTML of the page
html = driver.page_source
soup = bs4(html, 'html.parser')
# Find the specific table you want
table = soup.find('table', {'class': 'data'})  # Use appropriate attributes to locate the table
# Use pandas to read the HTML table
df = pd.read_html(str(table), header=1, skiprows=0, extract_links='body')[0]  # [0] to get the first DataFrame if there are multiple
# Extract the first element of each tuple in the MultiIndex
new_header = [tup[0] for tup in df.columns]
# Set the new header
df.columns = new_header
# Apply a lambda function to each column to extract the first element of each tuple
df = df.apply(lambda col: [v[0] if v[1] is None else v for v in col])
# Extract the player name and ID
df['player'] = df['Name'].apply(lambda x: x[0])
df['id'] = df['Name'].apply(lambda x: x[1])
# Define the regex pattern
pattern = r'^(?P<FullName>[A-Za-z\.\'\-\s]+) (?P<Positions>[A-Z0-9,]+) • (?P<Team>[A-Z]+)$'
# Apply the regex pattern to the DataFrame
df[['CBSNAME', 'Positions', 'Team']] = df['player'].str.extract(pattern)
# Split the FullName into FirstName and LastName
df[['FirstName', 'LastName']] = df['CBSNAME'].str.rsplit(n=1, expand=True)
# Extract CBSID
df['cbsid'] = df['id'].apply(lambda x: int(x.replace('/players/playerpage/', '')))
# Change dollar values column to int
for col in ['Mixed', 'AL-Only', 'NL-Only']:
    df[col] = df[col].apply(lambda x: int(x.replace('$','').replace('-','0')))
# Rename the Mixed column
df.rename(columns={'Mixed':'CBS'},inplace=True)
# Output the DataFrame to a CSV in the data/ folder
cols = ['cbsid','CBSNAME','FirstName','LastName','Positions','Pos','Team','CBS','AL-Only','NL-Only','Name','player','id']
df[cols].to_csv(f'{destination_path}/{datetime.now().year}-cbs-auction-values.csv', index=False)

C:\Users\pddnh\AppData\Local\Temp\ipykernel_64860\3081761616.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=1, skiprows=0, extract_links='body')[0]  # [0] to get the first DataFrame if there are multiple


In [12]:
df.sort_values('CBS', ascending=False)

,Name,Pos,Team,CBS,AL-Only,NL-Only,player,id,CBSNAME,Positions,FirstName,LastName,cbsid
0,"(Shohei Ohtani DH,SP • LAD, /players/playerpag...",DH,LAD,45,0,46,"Shohei Ohtani DH,SP • LAD",/players/playerpage/2901324,Shohei Ohtani,"DH,SP",Shohei,Ohtani,2901324
1,"(Bobby Witt SS • KC, /players/playerpage/3117472)",SS,KC,45,45,0,Bobby Witt SS • KC,/players/playerpage/3117472,Bobby Witt,SS,Bobby,Witt,3117472
2,"(Aaron Judge OF • NYY, /players/playerpage/207...",CF,NYY,44,44,0,Aaron Judge OF • NYY,/players/playerpage/2071264,Aaron Judge,OF,Aaron,Judge,2071264
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",3B,CLE,40,41,0,Jose Ramirez 3B • CLE,/players/playerpage/2044495,Jose Ramirez,3B,Jose,Ramirez,2044495
4,"(Elly De La Cruz SS • CIN, /players/playerpage...",SS,CIN,38,0,43,Elly De La Cruz SS • CIN,/players/playerpage/26912505,Elly De La Cruz,SS,Elly De La,Cruz,26912505
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,"(Lucas Giolito SP • BOS, /players/playerpage/2...",SP,BOS,0,3,0,Lucas Giolito SP • BOS,/players/playerpage/2000027,Lucas Giolito,SP,Lucas,Giolito,2000027
529,"(Robert Garcia RP • TEX, /players/playerpage/2...",RP,TEX,0,3,0,Robert Garcia RP • TEX,/players/playerpage/2836902,Robert Garcia,RP,Robert,Garcia,2836902
528,"(Dalton Rushing C • LAD, /players/playerpage/2...",C,LAD,0,0,3,Dalton Rushing C • LAD,/players/playerpage/29075514,Dalton Rushing,C,Dalton,Rushing,29075514
527,"(Tommy Kahnle RP • DET, /players/playerpage/20...",RP,DET,0,3,0,Tommy Kahnle RP • DET,/players/playerpage/2040766,Tommy Kahnle,RP,Tommy,Kahnle,2040766
